In [5]:
import medusa
import pandas as pd

from pickle import load

from medusa.flux_analysis.deletion import ensemble_single_gene_deletion

from cobra.flux_analysis.deletion import single_gene_deletion

In [6]:
# load the previously generated ensemble
with open('../results/ensembles/psy_ensemble_500_SEED_newbiomass.pickle', 'rb') as infile:
    ensemble = load(infile)

In [7]:
ensemble.base_model.objective.expression
ensemble.members

[<Member PST_gapfilled_0 at 0x125112c18>,
 <Member PST_gapfilled_21 at 0x125112c50>,
 <Member PST_gapfilled_22 at 0x125112c88>,
 <Member PST_gapfilled_83 at 0x125112cc0>,
 <Member PST_gapfilled_84 at 0x125112cf8>,
 <Member PST_gapfilled_114 at 0x125112d30>,
 <Member PST_gapfilled_115 at 0x125112d68>,
 <Member PST_gapfilled_118 at 0x125112da0>,
 <Member PST_gapfilled_119 at 0x125112dd8>,
 <Member PST_gapfilled_184 at 0x125112e10>,
 <Member PST_gapfilled_185 at 0x125112e48>,
 <Member PST_gapfilled_203 at 0x125112e80>,
 <Member PST_gapfilled_204 at 0x125112eb8>,
 <Member PST_gapfilled_243 at 0x125112ef0>,
 <Member PST_gapfilled_244 at 0x125112f28>,
 <Member PST_gapfilled_246 at 0x125112f60>,
 <Member PST_gapfilled_247 at 0x125112f98>,
 <Member PST_gapfilled_290 at 0x125112fd0>,
 <Member PST_gapfilled_291 at 0x125131048>,
 <Member PST_gapfilled_425 at 0x125131080>,
 <Member PST_gapfilled_426 at 0x1251310b8>,
 <Member PST_gapfilled_481 at 0x1251310f0>,
 <Member PST_gapfilled_482 at 0x125131

In [8]:
# make sure all exchange reactions are open to calculate
# growth rate in complete media
ex_rxns = [rxn for rxn in ensemble.base_model.reactions if
              rxn.id.startswith('EX_')]
for rxn in ex_rxns:
    rxn.lower_bound = -1000

# identify genes that are essential in all ensemble members by opening 
# all reactions and testing for essentiality

# Open all reactions in the ensemble
for feature in ensemble.features:
    component = ensemble.base_model.reactions.get_by_id(feature.base_component.id)
    component.lower_bound = -1000
    component.upper_bound = 1000

# Perform the essentiality simulations with all exchanges open in the base model
always_essential = single_gene_deletion(ensemble.base_model)

# Filter by always being essential
always_essential = always_essential.loc[always_essential['growth'] < 1E-6]
always_essential.index = [list(gene)[0] for gene in always_essential.index]
always_essential_genes = [ensemble.base_model.genes.get_by_id(g) 
                          for g in always_essential.index]
genes_to_ko = [g for g in ensemble.base_model.genes if g not in always_essential_genes]

# Perform the knockouts for genes that weren't essential in the opened base model
gene_knockout_results = ensemble_single_gene_deletion(ensemble, specific_genes = genes_to_ko)
knockout_df = pd.DataFrame()
for member in gene_knockout_results:
    gene_knockout_results[member].index = [list(gene)[0] for gene in gene_knockout_results[member].index]
    knockout_df[member] = gene_knockout_results[member]['growth']

knockout_df.to_csv('../results/ensemble_gene_knockouts_newbiomass.tsv', sep = '\t')

performing deletions for PST_gapfilled_247
performing deletions for PST_gapfilled_482
performing deletions for PST_gapfilled_426
performing deletions for PST_gapfilled_118
performing deletions for PST_gapfilled_291
performing deletions for PST_gapfilled_83
performing deletions for PST_gapfilled_481
performing deletions for PST_gapfilled_119
performing deletions for PST_gapfilled_114
performing deletions for PST_gapfilled_203
performing deletions for PST_gapfilled_185
performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_243
performing deletions for PST_gapfilled_425
performing deletions for PST_gapfilled_184
performing deletions for PST_gapfilled_244
performing deletions for PST_gapfilled_204
performing deletions for PST_gapfilled_84
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_290
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_246
performing deletions for PST_gapfilled_115
